Rode o notebook no Google Colab<br><a href="https://colab.research.google.com/github/p-alcadi/logistica-python/blob/main/logistica_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
## Problema do Caixeiro-Viajante (PCV)
# O PCV visa determinar a menor rota entre localidades, sendo crucial para setores como de logística. Este projeto tem como objetivo implementar em Python métodos de solução do PCV e compará-los graficamente.

# Nota: Esta é uma expansão de um projeto anterior, no qual usei Nearest Neighbor em Excel e VBA para resolver um problema logístico durante o processo seletivo da Voll Júnior, onde sou Diretor de Projetos.

# ✅ Importar a base de dados de endereços do Kaggle;
# ✅ Transformar a base de dados;
# ✅ Converter os endereços para coordenadas geográficas;
# ✅ Calcular as distâncias geográficas entre cada par de endereços;
# ✅ Montar a matriz de distâncias (matriz quadrada contendo as distâncias entre todos os pares);
# ✅ Solucionar com o método Brute Force;
# ✅ Solucionar com o método Nearest Neighbor (NN);
# [ ] Solucionar com o método Branch and Bound;
# [ ] Solucionar com o método do Algoritmo de Christofides;
# [2/4] Comparar graficamente cada solução.

In [12]:
# Importação das bilbiotecas necessárias
import csv
from timeit import default_timer as timer
from itertools import permutations
import kagglehub
import requests
import time
import urllib.parse
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd

In [13]:
######################################## Funções ################################################
# Define uma função para buscar as coordenadas de cada endereço na API Nominatim
def geocode(address):
    headers = {"User-Agent": "Mozilla/5.0 (compatible; AcmeInc/1.0)"}
    api_url = "https://nominatim.openstreetmap.org/search?q=" + urllib.parse.quote_plus(address) + "&format=json"
    response = requests.get(api_url, headers=headers)

    # Retorna uma exception caso a API não envie uma status de OK.
    try:
        response.raise_for_status()
    except requests.exceptions.HTTPError as error:
        print(str(error))
        return "Error:" + str(error)

    # A API retorna um JSON contendo todas as coordenadas compatíveis com o endereço fornecido.
    # A entrada [0] é o endereço mais próximo do fornecido.
    json = response.json()

    # Dormindo por 1.1 segundo para respeitar a política de uso da API Nominatim (máximo de um acesso por segundo)
    time.sleep(1.1)

    # Verifica se o json está vazio.
    # Caso esteja, significa que a API não encontrou nenhuma coordenada para o endereço fornecido.
    if not json:
        # Caso a API não encontrar o endereço fornecido, retorna False.
        return False
    else:
        # Caso a API encontrar o endereço fornecido, retorna um vetor com as coordenadas do primeiro resultado da busca.
        return ",".join([json[0]["lon"], json[0]["lat"]])

# Função que usa a API OSRM para buscar a matriz de distâncias de uma lista de endereços
def get_dmatrix(addresses):
    headers = {"User-Agent": "Mozilla/5.0 (compatible; AcmeInc/1.0)"}
    api_url = "http://router.project-osrm.org/table/v1/driving/" + ";".join(addresses) + "?annotations=distance"
    response = requests.get(api_url, headers=headers)

    # Retorna uma exception caso a API não envie uma status de OK.
    try:
        response.raise_for_status()
    except requests.exceptions.HTTPError as error:
        return "Error:" + str(error)

    return response.json()

# Função para calcular a distância total de uma rota, tomada como um vetor contendo, em ordem, os índices de cada endereço
# Exemplo: o vetor [5, 3, 4, 1, 7, 0, 2, 6] representa a rota que vai do endereço 5 para o 3, para o 4 e assim por diante
def total_route_dist(route):
    total_dist = 0
    size = len(route)

    for i in range(size):
        current_address = route[i]
        next_address = route[(i + 1) % size]
        total_dist += dist_dict[current_address][next_address]

    return total_dist

In [14]:
# Importa o dataset de endereços do Kagglehub
# Colunas: address, city, state, zip
path = kagglehub.dataset_download("ahmedshahriarsakib/list-of-real-usa-addresses", path="list_of_real_usa_addresses.csv")
data = []
with open(path) as file:
    reader = csv.reader(file)
    for line in reader:
        data.append(line)

# Filtra os endereços, selecionando apenas os de Connecticut
data_ct = [x for x in data if x[2] == "CT"]

# Reduz o dataset para economizar tempo
data_ct = data_ct[0:13]

# Para cada linha, concatena as colunas em uma só string e alimenta na função geocode, adicionando o valor retornado em uma nova coluna
data_ct = [x + [geocode(", ".join([y for y in x]))] for x in data_ct]

# Remove os endereços não encontrados
data_ct = [x for x in data_ct if x[4] != False]

# Separa a coluna coords em uma coluna de longitudes e uma coluna de latitudes
data_ct = [x + x[4].split(",") for x in data_ct]

for row in data_ct:
    print(row)

['255 W Main St', 'Avon', 'CT', '6001', '-72.8525413,41.8106959', '-72.8525413', '41.8106959']
['120 Commercial Parkway', 'Branford', 'CT', '6405', '-72.83094094104487,41.28122705', '-72.83094094104487', '41.28122705']
['1400 Farmington Ave', 'Bristol', 'CT', '6010', '-72.8969051,41.6974111', '-72.8969051', '41.6974111']
['161 Berlin Road', 'Cromwell', 'CT', '6416', '-72.71182941157579,41.60546575', '-72.71182941157579', '41.60546575']
['656 New Haven Ave', 'Derby', 'CT', '6418', '-73.05204965554864,41.313733049999996', '-73.05204965554864', '41.313733049999996']
['69 Prospect Hill Road', 'East Windsor', 'CT', '6088', '-72.6076253,41.9232537', '-72.6076253', '41.9232537']
['150 Gold Star Hwy', 'Groton', 'CT', '6340', '-72.06496641715536,41.36986815', '-72.06496641715536', '41.36986815']
['900 Boston Post Road', 'Guilford', 'CT', '6437', '-72.67932773780274,41.288426230185785', '-72.67932773780274', '41.288426230185785']
['2300 Dixwell Ave', 'Hamden', 'CT', '6514', '-72.9184196,41.37393

In [15]:
# Usa a função get_dmatrix para buscar a matriz de distâncias dos endereços fornecidos e transforma em um dataframe
matrix_json = get_dmatrix([x[4] for x in data_ct])
dist_dict  = {}

# Guarda a matrix de distâncias em um dicionário, para guardas os índices de cada ponto
i = 0
for row in matrix_json["distances"]:
  # Para cada linha, transforma o zero da diagonal principal em infinito, para facilitar os cálculos, e guarda no dicionário
  dist_dict[i] = [float("inf") if dist == 0 else dist for dist in row]
  i += 1

for key in dist_dict:
  print(dist_dict[key])

[inf, 72284.5, 17760.2, 38481.9, 69106.3, 37127.3, 93659.5, 80746.8, 54787.2, 18293.8, 90595, 31613.4]
[72580.9, inf, 58522.9, 49800.2, 22236.6, 89354.9, 70612.6, 14860.5, 18920.9, 65140.1, 94117.7, 80060.6]
[17504.2, 58475.7, inf, 24009.8, 55297.5, 44186.9, 99827.5, 66938, 40978.3, 18183.3, 96834.4, 37852.7]
[39037.6, 49782.9, 23107.7, inf, 48184.7, 42143.3, 75710.9, 47622.3, 34754.4, 19148, 83517.2, 32849]
[70195.6, 21911.8, 56137.6, 48840, inf, 88394.7, 89954.8, 34202.6, 15260.3, 64196.9, 113459.9, 78331.1]
[37523.2, 90399.7, 45487.9, 42674.5, 88801.5, inf, 97817.2, 88239.1, 75371.2, 26803, 94752.8, 18099.4]
[93668.7, 70717.3, 98729.6, 75565.6, 90225, 96773.5, inf, 57483.7, 86909.2, 81757.7, 37770.6, 77033.3]
[81487.4, 15834.3, 67429.4, 48667.3, 35342, 81474.2, 56785.1, inf, 32026.3, 64007.2, 80290.2, 72179.9]
[54820.3, 19002.6, 40762.3, 34376.7, 14728.6, 73931.3, 87045.6, 31293.4, inf, 49733.5, 98374.5, 63867.7]
[18319.2, 65162.3, 20576.5, 19404.3, 63531.9, 26036.7, 81748.7, 63001.

In [16]:
# cria um dicionário para guardar os resultados de cada método de solução
solutions = {}

In [17]:
############################################## Brute Force #####################################################
# O método Brute Force é o mais simples e é o único que rende uma solução exata.
# Entretanto, por requerer o cálculo de todas as rotas possíveis, é o método menos eficiente.
# O tempo de resolução cresce fatorialmente a cada endereço adicional.

solutions["brute_force"] = {}
solutions["brute_force"]["calculated_route"] = []
solutions["brute_force"]["route_dist"] = []
solutions["brute_force"]["time_taken"] = []
solutions["brute_force"]["error"] = []

starting_point = 2

for key in dist_dict:
    if key >= 3:
        # Define uma "menor distância" infinita para fins de comparação com as distâncias totais de cada rota
        # A cada comparação em que a distância total da rota é menor do que a "menor distância", a distância mínima recebe o valor da distâncai total da rota
        menor_dist = float("inf")
        route = None

        # Obtém todas os vetores de todas as permutações entre endereços, representando cada rota diferente
        perms = permutations(list(range(key)))

        # Para cada permutação em perms, compara a distância total da rota com a menor_dist
        # Se a distância total da rota for menor do que menor_dist, menor_dist recebe o valor da distância total da rota
        # O objetivo disso é encontrar a rota que tem a menor distância total e guardar no dicionário de resultados
        start = timer()
        for perm in perms:
            if perm[0] == starting_point:
              route_dist = total_route_dist(perm)
              if route_dist < menor_dist:
                  menor_dist = route_dist
                  route = perm
        end = timer()

        solutions["brute_force"]["calculated_route"].append(list(route))
        solutions["brute_force"]["route_dist"].append(menor_dist / 1000)
        solutions["brute_force"]["time_taken"].append(end - start)
        solutions["brute_force"]["error"].append(0)


In [18]:
################################## Nearest Neighbor #########################################
starting_point = 2

solutions["nn"] = {}
solutions["nn"]["calculated_route"] = []
solutions["nn"]["route_dist"] = []
solutions["nn"]["time_taken"] = []
solutions["nn"]["error"] = []

for i in range(len(dist_dict)):
  if i >= 3:
    truncated_dict = dict(list(dist_dict.items())[:i])

    for row_i in truncated_dict:
      truncated_dict[row_i] = [item for item_i, item in enumerate(truncated_dict[row_i]) if item_i < i]

    curr_point = starting_point
    route = []
    route.append(curr_point)
    total_dist = 0

    start = timer()
    while len(route) < len(truncated_dict):
      nearest_p = None
      nearest_dist = float("inf")
      for destination, dist in enumerate(truncated_dict[curr_point]):
        if destination not in route:
          if dist < nearest_dist:
            nearest_dist = dist
            curr_point = destination

      total_dist += nearest_dist
      route.append(curr_point)
    end = timer()

    total_dist += dist_dict[route[-1]][route[0]]

    solutions["nn"]["calculated_route"].append(route)
    solutions["nn"]["route_dist"].append(total_dist / 1000)
    solutions["nn"]["time_taken"].append(end - start)
    solutions["nn"]["error"].append((solutions["nn"]["route_dist"][i - 3] - solutions["brute_force"]["route_dist"][i - 3]) * 100 / solutions["brute_force"]["route_dist"][i - 3])

In [19]:
df_time = pd.DataFrame({"Brute Force": solutions["brute_force"]["time_taken"],
                   "NN": solutions["nn"]["time_taken"]}, [i for i in range(3, len(solutions["brute_force"]["time_taken"]) + 3)])

df_error = pd.DataFrame({"Brute Force": solutions["brute_force"]["error"],
                   "NN": solutions["nn"]["error"]}, [i for i in range(3, len(solutions["brute_force"]["error"]) + 3)])

fig = make_subplots(rows=1, cols=2)

fig_time = px.line(df_time,
                   title="Tempo de resolução de cada método",
                   labels = {"value": "Tempo (s)", "index": "Número de pontos", "variable": "Método"},
                   width=600,
                   height=350)

fig_error = px.line(df_error,
                    title="Erro relativo de cada método",
                    labels = {"value": "Erro relativo (%)", "index": "Número de pontos", "variable": "Método"},
                    width=600,
                    height=350)

fig_time.show()
fig_error.show()

In [20]:
################################## Branch and Bound #########################################

#reduction_elements = []

# Em cada linha, subtrai de todos as distâncias a distância mínima daquela linha
# E registra todas as distâncias minimas em `reduction_elements`
#row_reduced = [[y - min(x) for y in x] for x in dist_matrix_inf]
#reduction_elements += [min(x) for x in dist_matrix_inf if min(x) != 0]
#transposed_row_reduced = [[row[i] for row in row_reduced] for i in range(len(row_reduced[0]))]

# Em cada coluna, subtrai de todos as distâncias a distância mínima daquela coluna
# E registra todas as distâncias minimas em `reduction_elements`
#reduction_elements += [min(x) for x in transposed_row_reduced if min(x) != 0]
#transposed_col_reduced = [[y - min(x) for y in x] for x in transposed_row_reduced]
#col_reduced = [[row[i] for row in transposed_col_reduced] for i in range(len(transposed_col_reduced[0]))]

#upper_bound = float("inf")
#lower_bound = sum(reduction_elements)

